in this note book we implement excercise 2.


we will be working interactivley:

load variables into global and interact in jupyter. only things work and are understood, do we move to classes and fucntions

# imports and setup

In [ ]:
import sys, os
sys.path.append(os.getcwd())

In [ ]:
import numpy as np
import faiss
from helpers import *
from pathlib import Path


## brief summary of methods in helpers

**Helper functions:**
- `decision_tree_demo()`: Demo of training/predicting with DecisionTreeClassifier
- `loading_random_forest()`: Shows how to initialize RandomForestClassifier with 300 trees, max_depth=6
- `loading_xgboost()`: Shows how to initialize XGBClassifier with 300 trees, max_depth=6, lr=0.1
- `plot_decision_boundaries(model, X, y, title)`: Visualizes classifier decision boundaries on 2D space
- `knn_examples(X_train, Y_train, X_test, Y_test)`: Demo of training/predicting with KNNClassifier
- `read_data_demo(filename)`: Reads CSV and returns numpy array + column names

## data overview

**Task:** Create paths to all CSV files (train.csv, validation.csv, test.csv, AD_test.csv). Print first few rows of train and AD_test. Print shapes of all datasets.

In [ ]:
Path.cwd()

Path('/app/data/IML_2025/ex2_files')

In [ ]:
list(Path.cwd().iterdir())

[Path('/app/data/IML_2025/ex2_files/AD_test.csv'),
 Path('/app/data/IML_2025/ex2_files/ex_2.py'),
 Path('/app/data/IML_2025/ex2_files/ex2.pdf'),
 Path('/app/data/IML_2025/ex2_files/knn.py'),
 Path('/app/data/IML_2025/ex2_files/validation.csv'),
 Path('/app/data/IML_2025/ex2_files/__MACOSX'),
 Path('/app/data/IML_2025/ex2_files/test.csv'),
 Path('/app/data/IML_2025/ex2_files/ex2_instructions.md'),
 Path('/app/data/IML_2025/ex2_files/helpers.py'),
 Path('/app/data/IML_2025/ex2_files/train.csv'),
 Path('/app/data/IML_2025/ex2_files/ex_2.ipynb'),
 Path('/app/data/IML_2025/ex2_files/__pycache__')]

In [ ]:
train_path = Path('train.csv')
val_path = Path('validation.csv')
test_path = Path('test.csv')
ad_test_path = Path('AD_test.csv')

train_path, val_path, test_path, ad_test_path

(Path('train.csv'),
 Path('validation.csv'),
 Path('test.csv'),
 Path('AD_test.csv'))

In [ ]:
train_data, train_cols = read_data_demo(train_path)
ad_test_data, ad_test_cols = read_data_demo(ad_test_path)

print(f"Train data first 5 rows:\n{train_data[:5]}\n")
print(f"Train columns: {train_cols}\n")
print(f"AD_test data first 5 rows:\n{ad_test_data[:5]}\n")
print(f"AD_test columns: {ad_test_cols}")

Train data first 5 rows:
[[-90.2232  33.1831  22.    ]
 [-91.3027  36.0799   1.    ]
 [-95.7114  32.6776  40.    ]
 [-93.5819  42.3102   9.    ]
 [-81.23    40.8873  32.    ]]

Train columns: ['long', 'lat', 'state']

AD_test data first 5 rows:
[[34.78   32.08  ]
 [35.2066 31.7784]
 [34.9992 32.8192]
 [34.8    31.95  ]
 [34.8864 32.0889]]

AD_test columns: ['long', 'lat']


# part 1: KNN iplementation

specification notes

Algorithm 1: kNN Classification

Data: Training dataset Xtrain = {(xi, yi)}N

i=1, where xi ∈ R2 is the

feature vector and yi corresponds to its class label; Test instance
ˆxtest; Number of neighbors k.

Result: Predicted class label ˆytest for the test instance.
for each training instance (xi, yi) do

Calculate the distance di between xi and xtest.

Sort the distances di in ascending order and select the first k instances.
for each class c do

Count the occurrences of c in the selected k instances.
Assign the class label ˆytest to the one with the highest count.
return ˆytest



In [ ]:

class KNNClassifier:
    def __init__(self, k, distance_metric='l2'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.Y_train = None

    def fit(self, X_train, Y_train):
        """
        Update the kNN classifier with the provided training data.

        Parameters:
        - X_train (numpy array) of size (N, d): Training feature vectors.
        - Y_train (numpy array) of size (N,): Corresponding class labels.
        """
        self.X_train = X_train.astype(np.float32)
        self.Y_train = Y_train
        d = self.X_train.shape[1]
        if self.distance_metric == 'l2':
            self.index = faiss.index_factory(d, "Flat", faiss.METRIC_L2)
        elif self.distance_metric == 'l1':
            self.index = faiss.index_factory(d, "Flat", faiss.METRIC_L1)
        else:
            raise NotImplementedError
        pass
        self.index.add(self.X_train)

    def predict(self, X):
        """
        Predict the class labels for the given data.

        Parameters:
        - X (numpy array) of size (M, d): Feature vectors.

        Returns:
        - (numpy array) of size (M,): Predicted class labels.
        """
        X = X.astype(np.float32)
        distances, indices = self.index.search(X, self.k)
        neighbor_labels = self.Y_train[indices]
        return np.array([np.bincount(row.astype(int)).argmax() for row in neighbor_labels])

    def knn_distance(self, X):
        """
        Calculate kNN distances for the given data. You must use the faiss library to compute the distances.
        See lecture slides and https://github.com/facebookresearch/faiss/wiki/Getting-started#in-python-2 for more information.

        Parameters:
        - X (numpy array) of size (M, d): Feature vectors.

        Returns:
        - (numpy array) of size (M, k): kNN distances.
        - (numpy array) of size (M, k): Indices of kNNs.
        """
        X = X.astype(np.float32)
        return self.index.search(X, self.k)[0]



things i need to understand:

how does faiss knn work? what happens in fit?
we need to be able to play with the internals


In [ ]:
knn = KNNClassifier(k=5, distance_metric='l2')
knn.fit(train_data[:, :2], train_data[:, 2])
knn.X_train.shape, knn.Y_train.shape

((24031, 2), (24031,))

so the function we cate baout is faiss.index.search.

I assume that index.add(ndarrary) creates some kind of idexible rid over all samples to allow efficient finding of neighbours. 

In [ ]:
?knn.index.search

Signature: knn.index.search(x, k, *, params=None, D=None, I=None, numeric_type=0)
Docstring:
Find the k nearest neighbors of the set of vectors x in the index.

Parameters
----------
x : array_like
    Query vectors, shape (n, d) where d is appropriate for the index.
    `dtype` must be float32.
k : int
    Number of nearest neighbors.
params : SearchParameters
    Search parameters of the current search (overrides the class-level params)
D : array_like, optional
    Distance array to store the result.
I : array_like, optional
    Labels array to store the results.

Returns
-------
D : array_like
    Distances of the nearest neighbors, shape (n, k). When not enough results are found
    the label is set to +Inf or -Inf.
I : array_like
    Labels of the nearest neighbors, shape (n, k).
    When not enough results are found, the label is set to -1
File:      ~/.local/lib/python3.12/site-packages/faiss/class_wrappers.py
Type:      method

In [ ]:
test_point = np.array([[35.0, 32.0]], dtype=np.float32)
distances, indices = knn.index.search(test_point, k=5)
print(f"Query point: {test_point}")
print(f"\nDistances shape: {distances.shape}")
print(f"Indices shape: {indices.shape}")
print(f"\nDistances: {distances}")
print(f"Indices: {indices}")

Query point: [[35. 32.]]

Distances shape: (1, 5)
Indices shape: (1, 5)

Distances: [[10574.498 10622.248 10675.835 10743.521 10768.449]]
Indices: [[ 2682  6092 23297  9682 21731]]


In [ ]:
neighbor_points = knn.X_train[indices[0]]
neighbor_labels = knn.Y_train[indices[0]]
print(f"Neighbor coordinates:\n{neighbor_points}")
print(f"\nNeighbor labels: {neighbor_labels}")

Neighbor coordinates:
[[-67.0183  44.9137]
 [-67.224   45.134 ]
 [-67.4293  45.5674]
 [-67.8903  44.5343]
 [-67.8682  45.6595]]

Neighbor labels: [18. 18. 18. 18. 18.]


and use `np.bincount` to get the counts of all labels. creates a bin up to max of largest in input.

In [ ]:
np.bincount(neighbor_labels.astype(int)).argmax()


np.int64(18)

and for an array input

In [ ]:
test_points = np.array([[35.0, 32.0], [34.5, 31.5], [36.0, 33.0], [35.5, 32.5], [34.0, 31.0]], dtype=np.float32)
distances, indices = knn.index.search(test_points, k=5)
neighbor_labels = knn.Y_train[indices]
predicted_labels = np.array([np.bincount(row.astype(int)).argmax() for row in neighbor_labels])
print(f"Test points shape: {test_points.shape}")
print(f"Distances shape: {distances.shape}")
print(f"Indices shape: {indices.shape}")
print(f"Neighbor labels shape: {neighbor_labels.shape}")
print(f"Predicted labels: {predicted_labels}")

In [ ]:
knn.predict(test_points)

array([18, 18, 18, 18, 18])

nwo help me impelement the knn class knn_distances:

        Parameters:
        - X (numpy array) of size (M, d): Feature vectors.

        Returns:
        - (numpy array) of size (M, k): kNN distances.
        - (numpy array) of size (M, k): Indices of kNNs.


    simply use fias on new data to get all the disatnces. 
    what might b more efficient

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Looking at your exploration above, you've already figured out the key part! Here's the implementation:

```python
def knn_distance(self, X):
    "Calculate kNN distances for the given data"
    X = X.astype(np.float32)
    return self.index.search(X, self.k)
```

This returns a tuple of (distances, indices) which matches your required output. The faiss index already has the efficient search structure built from `.fit()`, so it's just a direct search call.

In [ ]:
def knn_distance(self, X):
    "Calculate kNN distances for the given data"
    X = X.astype(np.float32)
    return self.index.search(X, self.k)[0]

now lets run  a train test run. trian the knn on train data, then predict on test and get test accuarcy

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
val_data, val_cols = read_data_demo(val_path)
X_val, Y_val = val_data[:, :2], val_data[:, 2]
knn = KNNClassifier(k=5, distance_metric='l2')
knn.fit(train_data[:, :2], train_data[:, 2])
preds = knn.predict(X_val.astype(np.float32))
(preds == Y_val).mean()
```

In [ ]:
val_data, val_cols = read_data_demo(val_path)
X_val, Y_val = val_data[:, :2], val_data[:, 2]
knn = KNNClassifier(k=5, distance_metric='l2')
knn.fit(train_data[:, :2], train_data[:, 2])
preds = knn.predict(X_val.astype(np.float32))
(preds == Y_val).mean()

np.float64(0.9717043941411452)

In [ ]:
import numpy as np

# 1. Identify the majority class in the training set
y_train_labels = train_data[:, 2].astype(int)
counts = np.bincount(y_train_labels)
majority_class = counts.argmax()
majority_frequency = counts.max() / len(y_train_labels)

print(f"Majority Class in Train: {majority_class}")
print(f"Frequency in Train: {majority_frequency:.2%}")

# 2. Calculate the accuracy on Test set if we ONLY predicted that class
# (This is the theoretical limit as k -> N)
baseline_accuracy = (Y_test == majority_class).mean()

print(f"Baseline (Majority Vote) Accuracy: {baseline_accuracy:.4f}")

# 3. Compare with your specific result for k=3000
print("\nYour KNN Results for large k:")
print(df[df['k'] == 3000][['k', 'metric', 'accuracy']])

Majority Class in Train: 40
Frequency in Train: 6.11%
Baseline (Majority Vote) Accuracy: 0.0613

Your KNN Results for large k:
      k metric  accuracy
8  3000     l1  0.401798
9  3000     l2  0.398136


**Grid search:** Train kNN for all combinations of k ∈ {1, 10, 100, 1000, 3000} × distance ∈ {L1, L2}. Create 5×2 table of test accuracies. Save all 10 models with their hyperparameters and test accuracies.

**Questions:**
1. What's the trend as k increases? Does it differ by distance metric?
2. Visualize decision boundaries for: (i) L2 with k_max (best accuracy), (ii) L2 with k_min (worst accuracy), (iii) L1 with k_max
   - (a) Compare k_max vs k_min with L2: why does k_max perform better?
   - (b) Compare L2 vs L1 with k_max: how does distance metric affect the space?

think about the problem - what would you expect?

In [ ]:
#params
knn_size = [1,10,100,1000,3000]
metric = ['l1','l2']

train_data, train_cols = read_data_demo(train_path)
import pandas as pd
results = []
for k in knn_size:
    for m in metric:
        knn = KNNClassifier(k=k, distance_metric=m)
        knn.fit(train_data[:, :2], train_data[:, 2])
        preds = knn.predict(X_test)
        acc = (preds == Y_test).mean()
        results.append(dict(k=k, metric=m, accuracy=acc))
df = pd.DataFrame(results)
df.pivot(index='k', columns='metric', values='accuracy')

metric,l1,l2
k,,
1,0.967044,0.968375
10,0.961718,0.957723
100,0.923103,0.920107
1000,0.745007,0.741678
3000,0.401798,0.398136


as excpected, beyond some point, as K grows, the classifier just predicts the most common labels overall (if there is an over-represented state, then with very large k that tends to be the reuslt). hence beyond 10, the model detriorates. furthermore, since this model will only struggle with cities near borders, it is unsurprising that k=1 performed best.

furthermore, l1 perforemed slightly better than l2. since each computes distance slightly differnetly, this is an artifact of the data. 

### KNN decision boundaries


**Task:** Use `plot_decision_boundaries()` helper to visualize 3 models on test data:
1. L2 distance, k = k_max (highest test accuracy)
2. L2 distance, k = k_min (lowest test accuracy)  
3. L1 distance, k = k_max

## Anomaly detection

**Task:** Use AD_test.csv as test set, train.csv as single class (ignore labels).
1. Find 5 nearest neighbors (L2 distance) for each AD_test sample
2. Sum the 5 distances = anomaly score per sample
3. Select top 50 samples with highest scores = anomalies, rest = normal
4. Plot: AD_test colored by prediction (blue=normal, red=anomaly) + train.csv in black with alpha=0.01

**Question:** What characterizes the anomalies? How do they differ from normal data?

# trees

## Understanding trees - interactive exploration

**Goal:** Interact with tree components to understand how predictions work.
- Create a simple tree, examine its structure (tree_.feature, tree_.threshold, tree_.children_left/right, tree_.value)
- Manually trace a prediction path for a sample point
- Understand how tree.predict() navigates the tree structure

**Visualization library:** `sklearn.tree.plot_tree()` or `graphviz` for tree visualization. For more advanced viz, use `dtreeviz` library.

In [ ]:
# Tree visualization series placeholder

## Visualizing tree complexity

**Series 1 - Fixed depth=1, varying leaves:**
Train and visualize trees with max_depth=1 and max_leaf_nodes ∈ {2, 4, 8, 24, 50, 100}

**Series 2 - Varying depth, controlled leaves:**
Train and visualize trees with max_depth ∈ {2, 4, 8, 16, 32, 50} and appropriate max_leaf_nodes to observe depth effects

**Goal:** Understand how depth vs leaf constraints affect tree structure and decision boundaries.

**Visualization:** Use `sklearn.tree.plot_tree()` for tree structure or `plot_decision_boundaries()` for geographic space.

## train 24 trees

**Task:** Train DecisionTreeClassifier for all combinations of max_depth ∈ {1, 2, 4, 6, 10, 20, 50, 100} × max_leaf_nodes ∈ {50, 100, 1000}. Save all 24 models with their hyperparameters (max_depth, max_leaf_nodes) and accuracies (train, validation, test).

## tree analysis

**Questions:**
1. Report test accuracy of tree with best validation accuracy
2. Does this tree generalize well? Is validation set sufficient for selection?
3. Are 50 leaf nodes enough for perfect accuracy with 50 states? Why/why not?
4. Visualize decision boundaries for best tree from Q1. What shapes do classes form?
5. Find best tree with exactly 50 leaf nodes. Visualize and compare to Q1 best tree.
6. Find best tree with max_depth ≤ 6. Visualize and compare space division to Q4.

think and research - how many leaves do trees? how many boundaries can a tree make? how many hyper planes?

ask chat gpt for deep think

## random forest

**Task:** Train RandomForestClassifier with n_estimators=300, max_depth=6. Visualize decision boundaries. 

**Question:** Is this model more expressive than single tree from Q1? How does visualization show this?

# bonus: boosted trees

**Bonus (5 pts):** Train XGBClassifier with n_estimators=300, max_depth=6, learning_rate=0.1. Report test accuracy and visualize predictions. Compare to random forest: how do predictions differ? Which performs better?

## interpret Ml explainable boosting classifier

instead of xgb trees, use the glassbox EBM.

In [ ]:
.